In [1]:
import pandas as pd

df = pd.read_csv('./administrative_region_eng.csv')
print(len(df))
df.head()

4473


,origin,translated
0,서울특별시,Seoul Metropolitan Government
1,서울,Seoul
2,서울시,Seoul City
3,서울특별시 종로구,"Jongno-gu, Seoul"
4,서울특별시 종로구 청운효자동,"Cheongwoon Hyoja-dong, Jongno-gu, Seoul"


In [2]:
df.isna().sum()

origin        0
translated    0
dtype: int64

In [3]:
from collections import defaultdict

each_region_match = defaultdict(str)
for origin, translated in df.values.tolist():
    each_origin = origin.split()
    each_translated = translated.split(', ')
    each_translated.reverse()
    for o, t in zip(each_origin, each_translated):
        each_region_match[o] = t

len(list(each_region_match.keys()))

3899

In [9]:
print(each_region_match['광진구'])
print(each_region_match['경기'])
print(each_region_match['수송동']) # ????

Gwangjin-gu
Gyeonggi
Transport-dong


In [36]:
import re

with open('./juso_eng_dobongcilorkr.txt', 'r') as f:
    juso = f.read().splitlines()

removeChar = ['(il)', '(i)', '(sam)', '(sa)', '(o)', '(yuk)', \
    '(chil)', '(pal)', '(gu)', '(sip)', '(sibil)', '(sibi)', '(sipsam)']
for r in removeChar:
    juso = [j.replace(r, '') for j in juso]
juso = [j.replace(' - ', ' ') for j in juso]
specialChar = '[<>()]'
juso = [re.sub(specialChar, '', j) for j in juso]
juso = [j.strip() for j in juso if j.strip()]

In [38]:
juso[-20:-10]

['일도동 Ildo-dong',
 '해안동 Haean-dong',
 '화북1동 Hwabuk 1-dong',
 '화북2동 Hwabuk 2-dong',
 '화북동 Hwabuk-dong',
 '회천동 Hoecheon-dong',
 '남제주군 Namjeju-gun',
 '남원읍 Namwon-eup',
 '대정읍 Daejeong-eup',
 '성산읍 Seongsan-eup']

In [42]:
print(len(list(each_region_match.keys())))
for j in juso:
    splited = j.split()
    name, eng_name = splited[0], ' '.join(splited[1:])
    each_region_match[name] = eng_name
print(len(list(each_region_match.keys())))

4182
6233


In [43]:
print(each_region_match['수송동'])

Susong-dong


In [45]:
import pickle

with open('./total_region.pickle', 'rb') as f:
    total_region = pickle.load(f)
len(total_region)

44759

In [2]:
# Mongo Setting
from dbConn.mongoConn import config

collection = config(db_name="location_match_db", collection_name="region_name_kor_en")

In [9]:
all_data = collection.find()

In [12]:
for i, data in enumerate(all_data):
    lower_name = data['eng_name'].lower()
    collection.update({"eng_name": data['eng_name']}, {"$set": {"eng_name_lower": lower_name}})

<ipython-input-12-ba138ba9c3f4>:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  collection.update({"eng_name": data['eng_name']}, {"$set": {"eng_name_lower": lower_name}})


In [13]:
collection.find_one({'eng_name_lower': 'seoul'})

{'_id': ObjectId('611dd94a13351183ec3aea10'),
 'kor_name': '서울특별시',
 'eng_name': 'Seoul',
 'eng_name_lower': 'seoul'}